In [1]:
%load_ext autoreload
%autoreload 2
import img_text_composition_models
from enum import Enum
import datasets
import torchvision
import torch
from main import load_dataset, create_model_and_optimizer
import test_retrieval
from tqdm import tqdm
import cv2
from utils.FilePickling import pkl_save, pkl_load
import numpy as np
from test_retrieval import test

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [24]:
def error_stats_word(testset, error_ids, words_list):
    """
    Calculate p(S=wrong|word in S)
    """
    
    test_queries = testset.get_test_queries()
    
    dict_verb_wrong = {word:0 for word in words_list}
    dict_all_verb = {word:0 for word in words_list}
    
    # Count all words
    for i, query in enumerate(test_queries):
        that_str = query["mod"]["str"] 
        words = that_str.split()
        
        # Count all words
        for r_word in words_list:
            if r_word in words:
                dict_all_verb[r_word] += 1
                
        # Count error words
        if i in error_ids:
            for r_word in words_list:
                if r_word in words:
                    try:
                        dict_verb_wrong[r_word] += 1
                    except:
                        dict_verb_wrong[r_word] = 1
            
    stats_dict = {}
    
    for word in words_list:
        stats_dict[word] = dict_verb_wrong[word] / dict_all_verb[word]
        
       

        
    return stats_dict

In [7]:
class css_base_dataset_opt:
    def __init__(self):
        self.dataset = "css3d"
        self.dataset_path = "../data/CSSDataset/CSS-vn-v2-segmenter.json" 

class css_augmented_dataset_opt:
    def __init__(self):
        self.dataset = "css3d"
        self.dataset_path = "../data/CSSDataset/CSS-vn-augmented-segmenter-v2.json" 
        
class phobert_model_opt:
    def __init__(self):
        self.dataset = "css3d"
        self.model = "tirg_phobert" 
        self.loss = "soft_triplet" 
        self.comment = "tirg_phobert"
        self.embed_dim = 512
        self.learning_rate = 1e-2
        self.weight_decay = 1e-6 
        self.f = ""
        self.learning_rate_decay_frequency = 99999999
        self.batch_size = 32
        self.num_epochs = 100
        self.n_epochs_valudations = 5
        self.loader_num_workers = 4
        self.pretrained_weights = "runs/Dec12_14-57-59_ai-servers-1tirgphobert_css-vn-v2-segmenter/latest_checkpoint.pth"

class base_model_opt:
    def __init__(self):
        self.dataset = "css3d"
        self.model = "tirg" 
        self.loss = "soft_triplet" 
        self.comment = "tirg_base"
        self.embed_dim = 512
        self.learning_rate = 1e-2
        self.weight_decay = 1e-6 
        self.f = ""
        self.learning_rate_decay_frequency = 99999999
        self.batch_size = 32
        self.num_epochs = 100
        self.n_epochs_valudations = 5
        self.loader_num_workers = 4
        self.pretrained_weights = "runs/Dec17_11-17-51_ai-servers-1tirg_css-vn-v2-segmenter/best_checkpoint.pth"        

trainset_base, testset_base = load_dataset(css_base_dataset_opt())
_, testset_augmented = load_dataset(css_augmented_dataset_opt())
texts = [t for t in trainset_base.get_all_texts()]

base_model, _ = create_model_and_optimizer(base_model_opt(), texts)
phobert_model, _ = create_model_and_optimizer(phobert_model_opt(), texts)

Reading dataset  css3d
trainset size: 19012
testset size: 19057
Reading dataset  css3d
trainset size: 19012
testset size: 19057
Creating model and optimizer for tirg
Pretrained model from runs/Dec17_11-17-51_ai-servers-1tirg_css-vn-v2-segmenter/best_checkpoint.pth
Creating model and optimizer for tirg_phobert


Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.


Pretrained model from runs/Dec12_14-57-59_ai-servers-1tirgphobert_css-vn-v2-segmenter/latest_checkpoint.pth


In [8]:
out, error_ids = test(phobert_model_opt(), phobert_model, testset_base)
out

100%|██████████| 19057/19057 [00:58<00:00, 325.64it/s]


[('recall_top1_correct_composition', 0.7663509996123387),
 ('recall_top5_correct_composition', 0.9441767735504236),
 ('recall_top10_correct_composition', 0.968101013457385),
 ('recall_top50_correct_composition', 0.9919698731793765),
 ('recall_top100_correct_composition', 0.9951265437226561)]

In [ ]:
get_dict_wrong_verb(testset_base, error_ids)

In [6]:
out, error_ids = test(phobert_model_opt(), phobert_model, testset_augmented)

100%|██████████| 19057/19057 [00:59<00:00, 322.16it/s]


In [10]:
report = error_stats_word(testset_augmented, error_ids, words_list = ["bé", "to"])
report 

{'bé': 0.20936395759717313, 'to': 0.23345049794961922}

In [29]:
np.round(1 - 0.23345049794961922,4)

0.7665

In [ ]:
error_ids[0]

In [ ]:
testset_base.get_all_texts()

# Base model

In [11]:
report, error_ids = test(base_model_opt(), base_model, testset_base)
report

100%|██████████| 19057/19057 [00:56<00:00, 334.94it/s]


[('recall_top1_correct_composition', 0.7818574514038877),
 ('recall_top5_correct_composition', 0.9499916929722545),
 ('recall_top10_correct_composition', 0.9720883867752118),
 ('recall_top50_correct_composition', 0.9930774768787728),
 ('recall_top100_correct_composition', 0.9962895276070222)]

In [13]:
error_stats_word(testset_base, error_ids, words_list = ["nhỏ", "lớn"])

{'nhỏ': 0.19081272084805653, 'lớn': 0.21719390743995312}

In [14]:
report, error_ids = test(base_model_opt(), base_model, testset_augmented)

100%|██████████| 19057/19057 [00:56<00:00, 337.62it/s]


In [ ]:
testset_augmented.get_test_queries()[:100]

In [25]:
error_stats_word(testset_augmented, error_ids, words_list = ["bé", "to"])

{'bé': 0.5742049469964664, 'to': 0.5694200351493849}

In [31]:
np.round(1- 0.5694200351493849, 4)

0.4306

In [20]:
def count_prob(n_babies):
    prob = 1 
    for i in range(n_babies):
        prob *= (365-i)/365
    return 1-prob

In [22]:
count_prob(25)

0.568699703969464

In [12]:
10/365

0.0273972602739726

In [13]:
365/365

1.0